In [111]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Hugging Face API Key:")
os.environ["HF_TOKEN"] = os.environ["OPENAI_API_KEY"]

HF_LLM_URL = "https://meqlao8jlyg1d5w2.us-east-1.aws.endpoints.huggingface.cloud" + "/v1/"
os.environ["OPENAI_BASE_URL"] = HF_LLM_URL

In [112]:
HF_EMBED_URL = "https://nu1wajtcvdq4rid8.us-east-1.aws.endpoints.huggingface.cloud"

from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

embedding_model_cats = HuggingFaceEndpointEmbeddings(
    model=HF_EMBED_URL,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HF_TOKEN"],
)
embedding_model_dogs = HuggingFaceEndpointEmbeddings(
    model=HF_EMBED_URL,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HF_TOKEN"],
)

In [113]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def splitter2(filename):
    reddit_thread = ""
    with open(filename, "r") as f:
        reddit_thread = f.read()

    split_txt = RecursiveCharacterTextSplitter(
        chunk_size = 250,
        chunk_overlap = 20
    ).split_text(reddit_thread)

    return split_txt

# print(len(split_txt))

In [114]:
from langchain_community.vectorstores import Qdrant

class Document:
    def __init__(self, page_content):
        self.page_content = page_content
        self.metadata = {}

# documents = [Document(chunk) for chunk in split_txt]

embeddings_dict_cats = {}
embeddings_dict_dogs = {}

for chunk in splitter2("cats.txt"):
  embeddings_dict_cats[chunk] = embedding_model_cats.embed_query(chunk)


for chunk in splitter2("dogs.txt"):
  embeddings_dict_dogs[chunk] = embedding_model_dogs.embed_query(chunk)

# for i in range(0, len(documents), 32):
#   if i == 0:
#     vectorstore = Qdrant.from_documents(
#         documents[i:i+32],
#         embedding_model,
#         location=":memory:",
#         collection_name="LangChain Blogs")
#     continue
#   vectorstore.add_documents(documents[i:i+32])

In [115]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec_1, vec_2):
  # print(f"Vector 1: {vec_1}")
  # print(f"Vector 2: {vec_2}")
  return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

def retrieve_context(query, embeddings_dict, embedding_model):
  query_vector = embedding_model.embed_query(query)
  max_similarity = -float('inf')
  closest_chunk = ""

  for chunk, chunk_vector in embeddings_dict.items():
    cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

    if cosine_similarity_score > max_similarity:
      closest_chunk = chunk
      max_similarity = cosine_similarity_score

  return closest_chunk

def simple_rag(query, past_chat, embeddings_dict, embedding_model, chat_chain):
  context = retrieve_context(query, embeddings_dict, embedding_model)

  # print(f"query: {query}, past_chat: {past_chat}, context: {context}")
  # print(f"chat_chain: {chat_chain}")
  response = chat_chain.invoke({"query" : query, "past_chat" : past_chat, "context": context})

  return_package = {
      "query" : query,
      "response" : response,
      "retriever_context" : context
  }

  return return_package

In [119]:
from langchain_openai import ChatOpenAI

hf_llm = ChatOpenAI(
    model="tgi",
)

from langchain_core.prompts import ChatPromptTemplate

def chat_chain_fac(position):
    topic = "cats vs dogs"
    system_template = f"You are debating {topic} with someone. You are in favour of {position}. Use your knowledge to make strong arguments and rebuttals."
    human_template = "{content}"


    HUMAN_TEMPLATE = """
    #Past Chat:
    {past_chat}

    QUERY:
    {query}
    """

    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human", HUMAN_TEMPLATE[:-1500])
    ])

    chat_chain = chat_prompt | hf_llm
    return chat_chain



In [120]:
past_chat = ""
cats_chat_chain = chat_chain_fac("cats")
dogs_chat_chain = chat_chain_fac("dogs")
cats_rag = simple_rag("Cats or Dogs?", past_chat, embeddings_dict_cats, embedding_model_cats, cats_chat_chain)
dogs_rag = simple_rag("Cats or Dogs?", past_chat, embeddings_dict_dogs, embedding_model_dogs, dogs_chat_chain)

past_chat += f"Human: Cats or Dogs?\n\n"
past_chat += f"Cats AI: {cats_rag['response'].content}\n\n"
past_chat += f"Dogs AI: {dogs_rag['response'].content}\n\n"



In [123]:
next_query = dogs_rag['response'].content

print(past_chat)

for _ in range(10):
    cats_rag = simple_rag(next_query, past_chat[:-512], embeddings_dict_cats, embedding_model_cats, cats_chat_chain)
    print(f"Cats AI: {cats_rag['response'].content}")
    past_chat += f"Cats AI: {cats_rag['response'].content}\n\n"
    next_query = cats_rag['response'].content
    

    dogs_rag = simple_rag(next_query, past_chat[:-512], embeddings_dict_dogs, embedding_model_dogs, dogs_chat_chain)
    print(f"Dogs AI: {dogs_rag['response'].content}")
    past_chat += f"Dogs AI: {dogs_rag['response'].content}\n\n"
    next_query = dogs_rag['response'].content

Human: Cats or Dogs?

Cats AI: Cats are superior pets for several reasons. First, they are low-maintenance, requiring less attention, grooming, and exercise compared to dogs. This makes them ideal for busy individuals and families with hectic schedules. Second, cats are independent, which allows them to be content staying alone for long periods, while dogs often require constant interaction and companionship.

Moreover, cats are naturally clean and odor-free, which is a significant advantage over dogs, who may have specific odor issues. Furthermore, cats have a

Dogs AI: I completely understand your affection for cats, but let me highlight some reasons why dogs make great pets:

1. Socialization: Dogs are inherently social animals and thrive when they have a human family or other dogs to interact with. This makes them great companions, always happy to see you and eager for affection.

2. Training: Dogs are highly trainable and can learn commands, tricks, and good behaviors. This allows